# Cybersecurity advanced
## Lab 11: CA


Links to Syllabus

- [Theoretical](https://hogenttin.github.io/cybersecurity-advanced/lesson-11/theory/)
- [Practical](https://hogenttin.github.io/cybersecurity-advanced/lesson-11/lab/)

Creating the CA

On the ISProuter
```bash
isprouter:~$ openssl genrsa -out ca.key 4096 
isprouter:~$ openssl req -x509 -new -nodes -key ca.key -sha256 -days 3650 -out ca.crt
```

```bash
isprouter:~$ openssl genrsa -out webserver.key 2048 
 
isprouter:~$ openssl req -new -key webserver.key -out webserver.csr
```

Needed information for certificates:

```bash
Country Name (2 letter code) [AU]:BE 
State or Province Name (full name) [Some-State]:Oost-Vlaanderen 
Locality Name (eg, city) []:Gent 
Organization Name (eg, company) [Internet Widgits Pty Ltd]:HOGENT 
Organizational Unit Name (eg, section) []:CyberAdvanced 
Common Name (e.g. server FQDN or YOUR name) []:www.cybersec.internal 
Email Address []:maarten.adriaenssens@student.hogent.be
```

```bash
touch san.cnf 
vi san.cnf
```
```bash
[req] 
distinguished_name = req_distinguished_name 
req_extensions = req_ext 
prompt = no 
 
[req_distinguished_name] 
CN = www.cybersec.internal 
 
[req_ext] 
subjectAltName = @alt_names 
 
[alt_names] 
DNS.1 = www.example.com
```

```bash
openssl req -new -key webserver.key -out webserver.csr -config san.cnf 
 
isprouter:~$ openssl x509 -req -in webserver.csr -CA ca.crt -CAkey ca.key -CAcreateserial -out webserver.crt -days 365 -sha256 -extfile san.cnf -extensions req_ext
```

On the web device

```bash
sudo yum install mod_ssl 
sudo vi /etc/httpd/conf/httpd.conf
```

File content:
```bash
ServerRoot "/etc/httpd" 
 
LoadModule proxy_module modules/mod_proxy.so 
LoadModule proxy_http_module modules/mod_proxy_http.so 
LoadModule ssl_module modules/mod_ssl.so 
 
<VirtualHost *:443> 
    ServerName www.cybersec.internal 
    DocumentRoot /var/www/html 
 
    SSLEngine on 
 
    SSLCertificateFile /etc/httpd/ssl/webserver.crt
        SSLCertificateKeyFile /etc/httpd/ssl/webserver.key 
    SSLCACertificateFile /etc/httpd/ssl/ca.crt 
 
    <Directory /var/www/html> 
        Options Indexes FollowSymLinks 
        AllowOverride All 
        Require all granted 
    </Directory> 
</VirtualHost> 
 
ProxyPass "/services" "http://localhost:9200" 
ProxyPassReverse "/services" "http://localhost:9200" 
ProxyPass "/cmd" "http://localhost:8000/" 
ProxyPassReverse "/cmd" "http://localhost:8000/" 
ProxyPass "/assets" "http://localhost:8000/assets" 
ProxyPassReverse "/assets" "http://localhost:8000/assets" 
ProxyPass "/exec" "http://localhost:8000/exec" 
ProxyPassReverse "/exec" "http://localhost:8000/exec" 
 
Include conf.modules.d/*.conf 
 
User apache 
Group apache 
 
ServerAdmin root@localhost 
 
<Directory /> 
    AllowOverride none 
    Require all denied 
</Directory> 
 
DocumentRoot "/var/www/html" 
 
<Directory "/var/www"> 
    AllowOverride None 
    Require all granted 
</Directory> 
 
<Directory "/var/www/html"> 
    Options Indexes FollowSymLinks 
    AllowOverride None 
    Require all granted 
</Directory> 
 
<IfModule dir_module> 
    DirectoryIndex index.html 
</IfModule> 
 
<Files ".ht*"> 
    Require all denied 
</Files> 
 
ErrorLog "logs/error_log"
LogLevel warn 
 
<IfModule log_config_module> 
    LogFormat "%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-
Agent}i\"" combined 
    LogFormat "%h %l %u %t \"%r\" %>s %b" common 
 
    <IfModule logio_module> 
      LogFormat "%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%
{User-Agent}i\" %I %O" combinedio 
    </IfModule> 
 
    CustomLog "logs/access_log" combined 
</IfModule> 
 
<IfModule alias_module> 
    ScriptAlias /cgi-bin/ "/var/www/cgi-bin/" 
</IfModule> 
 
<Directory "/var/www/cgi-bin"> 
    AllowOverride None 
    Options None 
    Require all granted 
</Directory> 
 
<IfModule mime_module> 
    TypesConfig /etc/mime.types 
    AddType application/x-compress .Z 
    AddType application/x-gzip .gz .tgz 
    AddType text/html .shtml 
    AddOutputFilter INCLUDES .shtml 
</IfModule> 
 
AddDefaultCharset UTF-8 
 
<IfModule mime_magic_module> 
    MIMEMagicFile conf/magic 
</IfModule> 
 
EnableSendfile on 
 
IncludeOptional conf.d/*.conf
```

```bash
sudo mkdir -p /etc/httpd/ssl 
sudo chown apache:apache /etc/httpd/ssl/* 
sudo chmod 600 /etc/httpd/ssl/* 
 
sudo semanage fcontext -a -t httpd_sys_content_t "/etc/httpd/ssl(/.*)?" 
sudo restorecon -Rv /etc/httpd/ssl/
sudo yum update openssl mod_ssl 
sudo systemctl restart httpd
```

Test
```bash
curl --cacert ca.crt https://www.cybersec.internal 
curl -k https://www.cybersec.internal
```

# Demo

Open site op kali

Show ca.crt
- isprouter
`openssl x509 -in ca.crt -text -noout`
- kali
`openssl x509 -in /home/kali/ca.crt -text -noout`


Multiple sites:
- add in san.cnf
- *.cybersec.internal instead of www.cybersec.internal